#**Dataset**

In [1]:
import numpy as np
import pandas as pd
import datetime

def rescale_data(data_order, data_message, data_level, time_window=0.1):
    start_time = 34201
    end_time = 57599
    data = np.concatenate((data_message[:-1, 0].reshape([-1, 1]), data_order[1:, :]), axis=1)
    data_new = np.empty([int((end_time - start_time) / time_window) + 1, int(4 * data_level)])
    time_scale = np.arange(start_time, end_time, time_window)
    k = 0
    for i in range(time_scale.shape[0]):
        while k < data.shape[0] and data[k, 0] < time_scale[i]:
            k = k + 1
        
        data_new[i] = data[k - 1, 1:]

    n, m = data_new.shape
    for i in range(int(m / 2)):
        data_new[:, 2 * i] = data_new[:, 2 * i] / 10000

    return data_new,time_scale

In [2]:
#level_5
columns = ['L1_ask_price','L1_ask_quantity','L1_bid_price','L1_bid_quantity','L2_ask_price','L2_ask_quantity','L2_bid_price','L2_bid_quantity','L3_ask_price','L3_ask_quantity','L3_bid_price','L3_bid_quantity','L4_ask_price','L4_ask_quantity','L4_bid_price','L4_bid_quantity','L5_ask_price','L5_ask_quantity','L5_bid_price','L5_bid_quantity']

#level_10
#columns = ['L1_ask_price','L1_ask_quantity','L1_bid_price','L1_bid_quantity','L2_ask_price','L2_ask_quantity','L2_bid_price','L2_bid_quantity','L3_ask_price','L3_ask_quantity','L3_bid_price','L3_bid_quantity','L4_ask_price','L4_ask_quantity','L4_bid_price','L4_bid_quantity','L5_ask_price','L5_ask_quantity','L5_bid_price','L5_bid_quantity','L6_ask_price','L6_ask_quantity','L6_bid_price','L6_bid_quantity','L7_ask_price','L7_ask_quantity','L7_bid_price','L7_bid_quantity','L8_ask_price','L8_ask_quantity','L8_bid_price','L8_bid_quantity','L9_ask_price','L9_ask_quantity','L9_bid_price','L9_bid_quantity','L10_ask_price','L10_ask_quantity','L10_bid_price','L10_bid_quantity']

def get_dataframe(name):
  # load the order and message book. We use the level 5 limit order book data.
  file_path = './'
  data_name = '{}_2012-06-21_34200000_57600000_'.format(name)
  data_level = 5
  data_order = np.loadtxt(file_path + data_name + 'orderbook_' + str(data_level) + '.csv', delimiter=',')
  data_message = np.loadtxt(file_path + data_name + 'message_' + str(data_level) + '.csv', delimiter=',')

  # set time window, forecast size, and look back range.
  time_window = 0.25

  # turn data into evenly spaced, then split the one day dataset into two half day datasets.
  evenly_spaced_data,time_scale = rescale_data(data_order, data_message, data_level, time_window)

  even_df = pd.DataFrame(evenly_spaced_data,columns=columns)
  date  = pd.DataFrame(time_scale,columns=['date'])

  #mid = (even_df['L1_ask_price'] + even_df['L1_bid_price']) /2
  #mid_price = pd.DataFrame(mid,columns=['mid_price'])

  #ask_bid = pd.DataFrame((even_df['L1_ask_price'],even_df['L1_bid_price']),columns=['ask_bid'])
  #ask_bid_q = pd.DataFrame((even_df['L1_ask_quantity'],even_df['L1_bid_quantity']),columns=['ask_bid_quantity'])
  
  #even_df['ask_bid_price'] = list(zip(even_df['L1_ask_price'],even_df['L1_bid_price']))
  #even_df['ask_bid_quantity'] = list(zip(even_df['L1_ask_quantity'],even_df['L1_bid_quantity']))
  #new_df = even_df[['ask_bid_price','ask_bid_quantity']]
  #even_df.drop(['ask_bid_price', 'ask_bid_quantity'], axis=1,inplace=True)

  df = pd.concat([date,even_df],axis=1)
  df.dropna(inplace=True)
  day = '2012-6-21 '
  df.date =[(day + str(datetime.timedelta(seconds=i))) for i in df.date.values]
  df['date']= pd.to_datetime(df['date'])
  df['tic'] = name
  return df


In [3]:
AAPL_df = get_dataframe('AAPL')
AMZN_df = get_dataframe('AMZN')
GOOG_df = get_dataframe('GOOG')
INTC_df = get_dataframe('INTC')
MSFT_df = get_dataframe('MSFT')

data = pd.concat([AAPL_df,AMZN_df,GOOG_df,INTC_df,MSFT_df], ignore_index=True)

data = data.sort_values(['date','tic'],ascending=True).reset_index(drop=True)

In [4]:
data.shape

(467960, 22)

In [5]:
#data.set_index('date',inplace=True)
data.head()

,date,L1_ask_price,L1_ask_quantity,L1_bid_price,L1_bid_quantity,L2_ask_price,L2_ask_quantity,L2_bid_price,L2_bid_quantity,L3_ask_price,...,L3_bid_quantity,L4_ask_price,L4_ask_quantity,L4_bid_price,L4_bid_quantity,L5_ask_price,L5_ask_quantity,L5_bid_price,L5_bid_quantity,tic
0,2012-06-21 09:30:01,585.87,100.0,585.74,100.0,585.89,100.0,585.71,18.0,585.93,...,44.0,585.94,300.0,585.69,20.0,585.98,400.0,585.65,5.0,AAPL
1,2012-06-21 09:30:01,223.86,100.0,223.84,100.0,223.95,60.0,223.75,74.0,224.00,...,2.0,224.24,20.0,223.60,15.0,224.25,100.0,223.51,100.0,AMZN
2,2012-06-21 09:30:01,579.79,210.0,579.19,75.0,579.89,25.0,579.12,100.0,579.95,...,41.0,579.98,5.0,578.99,1.0,580.00,2937.0,578.70,400.0,GOOG
3,2012-06-21 09:30:01,27.53,326.0,27.52,200.0,27.54,673.0,27.51,3215.0,27.55,...,11781.0,27.56,200.0,27.49,400.0,27.57,1100.0,27.48,3761.0,INTC
4,2012-06-21 09:30:01,31.00,28256.0,30.98,100.0,31.01,1599.0,30.97,353.0,31.02,...,218.0,31.03,3400.0,30.95,8261.0,31.04,6767.0,30.94,1573.0,MSFT


#**installing & Importing**

In [6]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Agg')
import datetime


from finrl.apps import config
# from finrl.neo_finrl.preprocessor.preprocessors import data_split
#from finrl.neo_finrl.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.drl_agents.stablebaselines3.models import DRLAgent
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
import sys
sys.path.append("../FinRL-Library")

/home/abdelmoez/.local/lib/python3.8/site-packages/pyfolio/pos.py:26: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  warnings.warn(


In [7]:
import os
if not os.path.exists("./" + config.DATA_SAVE_DIR):
    os.makedirs("./" + config.DATA_SAVE_DIR)
if not os.path.exists("./" + config.TRAINED_MODEL_DIR):
    os.makedirs("./" + config.TRAINED_MODEL_DIR)
if not os.path.exists("./" + config.TENSORBOARD_LOG_DIR):
    os.makedirs("./" + config.TENSORBOARD_LOG_DIR)
if not os.path.exists("./" + config.RESULTS_DIR):
    os.makedirs("./" + config.RESULTS_DIR)

#**Feautre Pipeline**

#**Design Environment**

In [8]:
import gym
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from gym import spaces
from gym.utils import seeding
from stable_baselines3.common.vec_env import DummyVecEnv
from copy import deepcopy
%matplotlib inline

# from stable_baselines3.common import logger


class StockTradingEnv(gym.Env):
    """A stock trading environment for OpenAI gym"""

    metadata = {"render.modes": ["human"]}

    def __init__(
        self,
        df,
        stock_dim,
        #hmax,
        initial_amount,
        #buy_cost_pct,
        #sell_cost_pct,
        reward_scaling,
        state_space,
        action_space,
        depth_list,
        #turbulence_threshold=None,
        #risk_indicator_col="turbulence",
        debt_amount = 0,
        make_plots=False,
        print_verbosity=10,
        time_frame=0,
        initial=True,
        previous_state=[],
        model_name="",
        mode="",
        shares_limit=500,
        position_limit=500,
        no_limit=1,
        iteration=""):
        self.shares_limit = shares_limit
        self.position_limit = position_limit
        self.no_limit = no_limit
        self.time_frame = time_frame
        self.df = df
        self.stock_dim = stock_dim
        #self.hmax = hmax
        self.initial_amount = initial_amount
        self.debt_amount = debt_amount
        #self.buy_cost_pct = buy_cost_pct
        #self.sell_cost_pct = sell_cost_pct
        self.reward_scaling = reward_scaling
        self.state_space = state_space
        self.action_space = action_space
        self.depth_list = depth_list
        self.action_space = spaces.Box(low= 1, high= 100, shape=(self.stock_dim*2,))
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(self.state_space,))
        self.data = self.df.loc[self.stock_dim*self.time_frame : self.stock_dim*(self.time_frame + 1)-1,:]
        self.terminal = False
        self.make_plots = make_plots
        self.print_verbosity = print_verbosity
        #self.turbulence_threshold = turbulence_threshold
        #self.risk_indicator_col = risk_indicator_col
        self.initial = initial
        self.previous_state = previous_state
        self.model_name = model_name
        self.mode = mode
        self.iteration = iteration
        # initalize state
        self.state = self._initiate_state()

        # initialize reward
        self.reward = 0
        #self.turbulence = 0
        #self.cost = 0
        self.trades = 0
        self.episode = 0
        # memorize all the total balance change
        self.asset_memory = [self.initial_amount-self.debt_amount]
        self.rewards_memory = []
        self.actions_memory = []
        self.date_memory = [self._get_date()]
        # self.reset()
        self._seed()
          
    def _sell_short_stock(self, index, action):

        sell_num_shares = min(action, self.state[index + 2 + 5 * self.stock_dim])

        sell_amount = (
            self.state[index + 2 + 4*self.stock_dim]
            * sell_num_shares
            )
        debt = (
            self.state[index + 2 + 2*self.stock_dim]
            * sell_num_shares
            )
        # update balance
        self.state[0] += sell_amount
        self.state[1] += debt
        self.state[index + 2+self.stock_dim] += sell_num_shares    
        self.trades += 1

        return sell_num_shares
            
    def _buy_short_stock(self, index, action):
        
        if self.state[2+index+self.stock_dim] <= 0:
            return 0
            
        else:
            available_amount = (self.state[0] // self.state[index + 2 + self.stock_dim*2])
            # print('available_amount:{}'.format(available_amount))

            # update balance
            buy_num_shares = min(action,available_amount, self.state[index+2+self.stock_dim],self.state[index + 2 + 3*self.stock_dim])

            buy_amount = (
                  self.state[index + 2 + self.stock_dim*2] * buy_num_shares 
                )
            
            self.state[0] -= buy_amount
            self.state[1] -= buy_amount
            self.state[index + 2+self.stock_dim] -= buy_num_shares
            self.trades += 1

            return buy_num_shares
            
    def _sell_stock(self, index, action):

            if self.state[2+index] <= 0:
              return 0
            
            else:
              sell_num_shares = min(action,self.state[2+index], self.state[index + 2 + 5 * self.stock_dim])

            sell_amount = (
                  self.state[index + 2 + 4*self.stock_dim]
                  * sell_num_shares
                )
            # update balance
            self.state[0] += sell_amount
            self.state[index + 2] -= sell_num_shares    
            self.trades += 1

            return sell_num_shares
            
    def _buy_stock(self, index, action):

            available_amount = (self.state[0] // self.state[index + 2 + self.stock_dim*2])
            # print('available_amount:{}'.format(available_amount))

            # update balance
            buy_num_shares = min(action,available_amount, self.state[index + 2 + 3*self.stock_dim])

            buy_amount = (
                  self.state[index + 2 + self.stock_dim*2] * buy_num_shares 
                )
            
            self.state[0] -= buy_amount
            self.state[index + 2] += buy_num_shares
            self.trades += 1

            return buy_num_shares



    def _make_plot(self):
        plt.plot(self.asset_memory, "r")
        plt.savefig("results/account_value_trade_{}.png".format(self.episode))
        plt.close()

    def step(self, actions):
        self.terminal = self.time_frame >= (len(self.df.date.unique())-1)
        self.debt_amount = sum(
            np.array(self.state[(self.stock_dim + 2):(self.stock_dim*2 + 2)])
            * np.array(self.state[(self.stock_dim * 2 + 2) : (self.stock_dim * 3 + 2)]))
        self.state[1] = self.debt_amount

        if self.terminal:
            # print(len(self.state))
            # print(len(self.state[1 : (self.stock_dim + 1)]))
            # print(len(self.state[(self.stock_dim * 3 + 1) : (self.stock_dim * 4 + 1)]))
            # print(f"Episode: {self.episode}")
            if self.make_plots:
                self._make_plot()
            end_total_asset = self.state[0] - self.state[1] + sum(
                np.array(self.state[2 : (self.stock_dim + 2)])
                * np.array(self.state[(self.stock_dim * 4 + 2) : (self.stock_dim * 5 + 2)])
            )
            df_total_value = pd.DataFrame(self.asset_memory)
            tot_reward = ( end_total_asset - self.initial_amount)
            df_total_value.columns = ["account_value"]
            df_total_value["date"] = self.date_memory
            df_total_value["daily_return"] = df_total_value["account_value"].pct_change(
                1
            )
            if df_total_value["daily_return"].std() != 0:
                sharpe = (
                    (252 ** 0.5)
                    * df_total_value["daily_return"].mean()
                    / df_total_value["daily_return"].std()
                )
            df_rewards = pd.DataFrame(self.rewards_memory)
            df_rewards.columns = ["account_rewards"]
            df_rewards["date"] = self.date_memory[:-1]
            if self.episode % self.print_verbosity == 0:
                print(f"Moment: {self.time_frame}, episode: {self.episode}")
                print(f"begin_total_asset: {self.asset_memory[0]:0.2f}")
                print(f"end_total_asset: {end_total_asset:0.2f}")
                print(f"total_reward: {tot_reward:0.2f}")
                #print(f"total_cost: {self.cost:0.2f}")
                print(f"total_trades: {self.trades}")
                if df_total_value["daily_return"].std() != 0:
                    print(f"Sharpe: {sharpe:0.3f}")
                print("=================================")

            if (self.model_name != "") and (self.mode != ""):
                df_actions = self.save_action_memory()
                df_actions.to_csv(
                    "results/actions_{}_{}_{}.csv".format(
                        self.mode, self.model_name, self.iteration
                    )
                )
                df_total_value.to_csv(
                    "results/account_value_{}_{}_{}.csv".format(
                        self.mode, self.model_name, self.iteration
                    ),
                    index=False,
                )
                df_rewards.to_csv(
                    "results/account_rewards_{}_{}_{}.csv".format(
                        self.mode, self.model_name, self.iteration
                    ),
                    index=False,
                )
                plt.plot(self.asset_memory, "r")
                plt.savefig(
                    "results/account_value_{}_{}_{}.png".format(
                        self.mode, self.model_name, self.iteration
                    ),
                    index=False,
                )
                plt.close()

            # Add outputs to logger interface
            # logger.record("environment/portfolio_value", end_total_asset)
            # logger.record("environment/total_reward", tot_reward)
            # logger.record("environment/total_reward_pct", (tot_reward / (end_total_asset - tot_reward)) * 100)
            # logger.record("environment/total_cost", self.cost)
            # logger.record("environment/total_trades", self.trades)
            # print(self.state[0:self.stock_dim+1])
            return self.state, self.reward, self.terminal, {}

        else:
            limit_dict = {
                'S_Stock' : 0,
                'S_Pos' : 0,
                'L_Stock' : 0,
                'L_Pos' : 0}
            limit_list = [limit_dict]*self.stock_dim
            
            #actions = actions * self.hmax  # actions initially is scaled between 0 to 1
            actions = actions.astype(
                int
            )  # convert into integer because we can't by fraction of shares
            # print(actions.shape)
            # print(actions)
            begin_total_asset = self.state[0] - self.state[1]+ sum(
                np.array(self.state[2 : (self.stock_dim + 2)])
                * np.array(self.state[(self.stock_dim * 4 + 2) : (self.stock_dim * 5 + 2)])
            )
            # print("begin_total_asset:{}".format(begin_total_asset))

            
            sell_index = np.intersect1d(np.where(actions[:self.stock_dim] > 60),np.where(actions[:self.stock_dim]<81)).tolist()
            buy_index = np.intersect1d(np.where(actions[:self.stock_dim] > 0),np.where(actions[:self.stock_dim]<21)).tolist()
            short_sell_id = np.intersect1d(np.where(actions[:self.stock_dim] > 80),np.where(actions[:self.stock_dim]<101)).tolist()
            short_buy_id = np.intersect1d(np.where(actions[:self.stock_dim] > 20),np.where(actions[:self.stock_dim]<41)).tolist()
            new_action = ['None']*self.stock_dim
            # print(buy_index)
            # print(sell_index)
            
            if not self.no_limit:
                for i in range(self.stock_dim):
                  if limit_list[i]['L_Stock'] >= 0:
                    limit_list[i]['L_Pos'] +=1
                  if limit_list[i]['S_Stock'] >= 0:
                    limit_list[i]['S_Pos'] +=1
                  if limit_list[i]['L_Stock']>=self.shares_limit:
                    if i in buy_index:
                      buy_index.remove(i)
                  if limit_list[i]['S_Stock']>=self.shares_limit:
                    if i in short_sell_id:
                      short_sell_id.remove(i)
                  if limit_list[i]['L_Pos']>=self.position_limit:
                    val = self._sell_stock(i, self.state[i+2])
                    new_action[i] = 'LS'+str(val)
                    limit_list[i]["L_Stock"] -= val
                    if limit_list[i]["L_Stock"] <= 0: 
                      limit_list[i]["L_Pos"] = 0
                    if i in buy_index:
                      buy_index.remove(i)
                  if limit_list[i]['L_Pos']>=self.position_limit:
                    val = self._buy_short_stock(i, self.state[i+2+self.stock_dim])
                    new_action[i] = 'SB'+str(val)
                    limit_list[i]["S_Stock"] -= val
                    if limit_list[i]["S_Stock"] <= 0: 
                      limit_list[i]["S_Pos"] = 0
                    if i in short_sell_id:
                      short_sell_id.remove(i)

            for index in sell_index:
                # print(f"Num shares before: {self.state[index+self.stock_dim+1]}")
                # print(f'take sell action before : {actions[index]}')
                val = self._sell_stock(index, actions[self.stock_dim:][index])
                new_action[index] = 'LS'+str(val)
                limit_list[index]["L_Stock"] -= val
                if limit_list[index]["L_Stock"] <= 0: 
                  limit_list[index]["L_Pos"] = 0
                # print(f'take sell action after : {actions[index]}')
                # print(f"Num shares after: {self.state[index+self.stock_dim+1]}")

            for index in buy_index:
                # print('take buy action: {}'.format(actions[index]))
                val = self._buy_stock(index, actions[self.stock_dim:][index])                                                                                      
                new_action[index] = 'LB'+str(val)
                limit_list[index]["L_Stock"] += val
            for index in short_sell_id:
                            # print('take buy action: {}'.format(actions[index]))
                val = self._sell_short_stock(index, actions[self.stock_dim:][index])
                new_action[index] = 'SS'+str(val)
                limit_list[index]["S_Stock"] += val
            for index in short_buy_id:
                            # print('take buy action: {}'.format(actions[index]))
                val = self._buy_short_stock(index, actions[self.stock_dim:][index])
                new_action[index] = 'SB'+str(val)
                limit_list[index]["S_Stock"] -= val
                if limit_list[index]["S_Stock"] <= 0: 
                  limit_list[index]["S_Pos"] = 0

            self.actions_memory.append(new_action)

            # state: s -> s+1
            self.time_frame += 1
            self.data = self.df.loc[self.stock_dim*self.time_frame : self.stock_dim*(self.time_frame + 1)-1,:]
            self.state = self._update_state()

            # print(len(self.state))
            # print(len(self.state[1 : (self.stock_dim + 1)]))
            # print(len(self.state[(self.stock_dim * 3 + 1) : (self.stock_dim * 4 + 1)]))

            end_total_asset = self.state[0] - self.state[1]+ sum(
                np.array(self.state[2 : (self.stock_dim + 2)])
                * np.array(self.state[(self.stock_dim * 4 + 2) : (self.stock_dim * 5 + 2)])
            )
            self.asset_memory.append(end_total_asset)
            self.date_memory.append(self._get_date())
            self.reward = end_total_asset - begin_total_asset
            self.rewards_memory.append(self.reward)
            self.reward = (self.reward * self.reward_scaling)
        # print(len(self.state))
        # print(self.state[1:self.stock_dim+1])
        # print(self.state[0])

        return self.state, self.reward, self.terminal, {}

    def reset(self):
        # initiate state
        self.state = self._initiate_state()

        if self.initial:
            self.asset_memory = [self.initial_amount-self.debt_amount]
        else:
            if self.stock_dim>1:
              previous_total_asset = self.previous_state[0] - self.state[1] + sum(
               np.array(self.state[2 : (self.stock_dim + 2)])
                * np.array(self.state[(self.stock_dim * 4+ 2) : (self.stock_dim * 5 + 2)])
                )
            else:
              previous_total_asset = self.previous_state[0] -self.state[1]+ self.state[2]*self.state[5]
            
            self.asset_memory = [previous_total_asset]

        self.time_frame = 0
        self.data = self.df.loc[self.stock_dim*self.time_frame : self.stock_dim*(self.time_frame + 1)-1,:]
        #self.turbulence = 0
        #self.cost = 0
        self.trades = 0
        self.terminal = False
        # self.iteration=self.iteration
        self.rewards_memory = []
        self.actions_memory = []
        self.date_memory = [self._get_date()]

        self.episode += 1
        return self.state

    def render(self, mode="human", close=False):
        return self.state

    def _initiate_state(self):
        if self.initial:
            # For Initial State
            if len(self.df.tic.unique()) > 1:
                # for multiple stock
                state = (
                    [self.initial_amount]
                    + [self.debt_amount]
                    #+ self.data.close.values.tolist()
                    + [0] * self.stock_dim
                    + [0] * self.stock_dim
                    + sum(
                        [
                            self.data[depth].values.tolist()
                            for depth in self.depth_list
                        ],
                        [],
                    )
                )
            else:
                # for single stock
                state = (
                    [self.initial_amount]
                    + [self.debt_amount]
                    #+ self.data.close.values.tolist()
                    + [0] * self.stock_dim
                    + [0] * self.stock_dim
                    + sum([[self.data[depth]] for depth in self.depth_list], [])
                )
        else:
            # Using Previous State
            if len(self.df.tic.unique()) > 1:
                # for multiple stock
                state = (
                    [self.previous_state[0]]
                    +[self.previous_state[1]]
                    #+ self.data.close.values.tolist()
                    + self.previous_state[
                        2 : (self.stock_dim*2 + 2)
                    ]
                    + sum(
                        [
                            self.data[depth].values.tolist()
                            for depth in self.depth_list
                        ],
                        [],
                    )
                )
            else:
                # for single stock
                state = (
                    [self.previous_state[0]]
                    + [self.previous_state[1]]
                    + self.previous_state[
                        2 : (self.stock_dim*2 + 2)
                    ]
                    + sum([[self.data[depth]] for depth in self.depth_list], [])
                )

        return state

    def _update_state(self):
        if len(self.df.tic.unique()) > 1:
            # for multiple stock
            state = (
                [self.state[0]]
                +[self.state[1]]
                + self.state[2 : (self.stock_dim*2 + 2)]
                + sum(
                    [
                        self.data[depth].values.tolist()
                        for depth in self.depth_list
                    ],
                    [],
                )
            )

        else:
            # for single stock
            state = (
                [self.state[0]]
                + [self.state[1]]
                + self.state[2 : (self.stock_dim + 2)]
                + sum([[self.data[depth]] for depth in self.depth_list], [])
            )

        return state

    def _get_date(self):
        if len(self.df.tic.unique()) > 1:
            date = self.data.date.unique()
        else:
            date = self.data.date
        return date
 
    def save_asset_memory(self):
        date_list = self.date_memory
        asset_list = self.asset_memory
        # print(len(date_list))
        # print(len(asset_list))
        df_account_value = pd.DataFrame(
            {"date": date_list, "account_value": asset_list}
        )
        return df_account_value

    def save_action_memory(self):
        if len(self.df.tic.unique()) > 1:
            # date and close price length must match actions length
            date_list = self.date_memory[:-1]
            df_date = pd.DataFrame(date_list)
            df_date.columns = ["date"]

            action_list = self.actions_memory
            df_actions = pd.DataFrame(action_list)
            df_actions.columns = self.data.tic.values
            df_actions.index = df_date.date
            # df_actions = pd.DataFrame({'date':date_list,'actions':action_list})
        else:
            date_list = self.date_memory[:-1]
            action_list = self.actions_memory
            df_actions = pd.DataFrame({"date": date_list, "actions": action_list})
        return df_actions

    def _seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def get_sb_env(self):
      def get_self():
          return deepcopy(self)
      e = DummyVecEnv([get_self])
      obs = e.reset()
      return e, obs

In [9]:
from math import floor
split = 0.80
split_date = data.date.unique()[:floor(split*data.date.unique().shape[0])]
train = data[data.date<split_date[-1]]
trade = data[data.date>split_date[-2]].reset_index(drop=True)

##**Environment for Portfolio Allocation**

In [10]:
stock_dimension = len(train.tic.unique())
state_space = 2 + (len(columns)+2)*(stock_dimension)
#state_space = 1 + 2*stock_dimension + len(tech_indicator_list)*stock_dimension
#state_space = 1 + 2*(stock_dimension+2) + (len(tech_indicator_list)-2)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 5, State Space: 112


In [11]:
env_kwargs = {
    "initial_amount": 100000,
    #"buy_cost_pct": 0.001,
    #"sell_cost_pct": 0.001,
    "state_space": state_space, 
    "stock_dim": stock_dimension, 
    "depth_list": columns, 
    "action_space": stock_dimension, 
    "reward_scaling": 1e-4,
    "shares_limit":1000,
    "position_limit":1000,
    "no_limit":0
}
e_train_gym = StockTradingEnv(df = train ,**env_kwargs)

In [12]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


In [13]:
e_trade_gym = StockTradingEnv(df = trade, **env_kwargs)

In [14]:
env_trade, _ = e_trade_gym.get_sb_env()
print(type(env_trade))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


#**DRL Algorithms**

In [15]:
def DRL_prediction(model, environment):
        test_env, test_obs = environment.get_sb_env()
        """make a prediction"""
        account_memory = []
        actions_memory = []
        test_env.reset()
        for i in range(len(environment.df.date.unique())):
            action, _states = model.predict(test_obs)
            # account_memory = test_env.env_method(method_name="save_asset_memory")
            # actions_memory = test_env.env_method(method_name="save_action_memory")
            test_obs, rewards, dones, info = test_env.step(action)
            if i == (len(environment.df.date.unique()) - 2):
                account_memory = test_env.env_method(method_name="save_asset_memory")
                actions_memory = test_env.env_method(method_name="save_action_memory")
            if dones[0]:
                print("hit end!")
                break
        return account_memory[0], actions_memory[0]

In [16]:
# initialize
agent = DRLAgent(env = env_train)
trade_agent = DRLAgent(env = env_trade)

##**A2C**

In [17]:
A2C_PARAMS = {'n_steps': 100, 'ent_coef': 0.02, 'learning_rate': 0.0007}
model_a2c = agent.get_model(model_name="a2c",model_kwargs = A2C_PARAMS)

{'n_steps': 100, 'ent_coef': 0.02, 'learning_rate': 0.0007}
Using cpu device


###**Train**

In [18]:
trained_a2c = agent.train_model(model=model_a2c, 
                                tb_log_name='a2c',
                                total_timesteps=train.date.unique().shape[0]*75
                                )

------------------------------------
| time/                 |          |
|    fps                | 23       |
|    iterations         | 100      |
|    time_elapsed       | 431      |
|    total_timesteps    | 10000    |
| train/                |          |
|    entropy_loss       | -14.8    |
|    explained_variance | -9.44    |
|    learning_rate      | 0.0007   |
|    n_updates          | 99       |
|    policy_loss        | -1.13    |
|    reward             | 0.0      |
|    std                | 1.07     |
|    value_loss         | 0.0211   |
------------------------------------
------------------------------------
| time/                 |          |
|    fps                | 23       |
|    iterations         | 200      |
|    time_elapsed       | 864      |
|    total_timesteps    | 20000    |
| train/                |          |
|    entropy_loss       | -15.6    |
|    explained_variance | -0.592   |
|    learning_rate      | 0.0007   |
|    n_updates          | 199      |
|

In [19]:
trained_a2c.save('./trained_models/a2cLOB1')

###**Trade**

In [20]:
df_daily_return, df_actions = DRL_prediction(model=trained_a2c,environment= e_trade_gym)

hit end!


In [21]:
df_daily_return

,date,account_value
0,[2012-06-21T14:41:59.000000000],100000.00
1,[2012-06-21T14:41:59.250000000],99999.30
2,[2012-06-21T14:41:59.500000000],99969.87
3,[2012-06-21T14:41:59.750000000],99961.52
4,[2012-06-21T14:42:00.000000000],99959.94
...,...,...
18715,[2012-06-21T15:59:57.750000000],-136241.57
18716,[2012-06-21T15:59:58.000000000],-133897.71
18717,[2012-06-21T15:59:58.250000000],-113868.77
18718,[2012-06-21T15:59:58.500000000],-111941.24


In [27]:
df_actions
#df_actions

,AAPL,AMZN,GOOG,INTC,MSFT
date,,,,,
2012-06-21 14:41:59.000,LB1,SS1,LB1,LB1,LB1
2012-06-21 14:41:59.250,LB100,LB100,LB1,SS1,LB100
2012-06-21 14:41:59.500,LB25.0,SS21.0,SS1,SS100,SS73
2012-06-21 14:41:59.750,LB1,SS1,LB1,SS100,LS100
2012-06-21 14:42:00.000,LB1,LB1,SS1,SS1,SS100
...,...,...,...,...,...
2012-06-21 15:59:57.500,LB1,SS100,LB1,SS1,LS1
2012-06-21 15:59:57.750,SS100,LB1,LB42.0,SS100,SS100
2012-06-21 15:59:58.000,LB100,SS100,SS100,LB100,LB100


In [23]:
df_actions['GOOG'].values

array(['LB1', 'LB1', 'SS1', ..., 'SS100', 'LB1', 'LB1'], dtype=object)

###**BackTest**

In [24]:
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from finrl.plot import backtest_plot, convert_daily_return_to_pyfolio_ts, get_daily_return
from pyfolio import timeseries
import pyfolio

In [25]:
df_daily_return['date']=pd.DataFrame(np.concatenate(df_daily_return.date.values))

In [26]:
print("==============A2C Strategy Stats===========")
perf_stats_all = backtest_stats(df_daily_return)

==============A2C Strategy Stats===========
Annual return                NaN
Cumulative returns     -2.721930
Annual volatility      45.626816
Sharpe ratio           -0.048493
Calmar ratio                 NaN
Stability                    NaN
Max drawdown           -3.295523
Omega ratio             0.855219
Sortino ratio          -0.050743
Skew                         NaN
Kurtosis                     NaN
Tail ratio              1.070354
Daily value at risk    -5.757219
dtype: float64


/home/abdelmoez/.local/lib/python3.8/site-packages/empyrical/stats.py:447: RuntimeWarning: invalid value encountered in double_scalars
  return ending_value ** (1 / num_years) - 1
/home/abdelmoez/.local/lib/python3.8/site-packages/empyrical/stats.py:1494: RuntimeWarning: invalid value encountered in log1p
  cum_log_returns = np.log1p(returns).cumsum()
